# Import necessary libraries


In [295]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import re

pd.options.display.max_columns = 200
pd.options.display.max_rows = 3000

# Load the dataset 

In [296]:
def parent_directory():

  # Create a relative path to the parent
  # of the current working directory
  relative_parent = os.path.join(os.getcwd(), os.pardir)
  # Return the absolute path of the parent directory
  return os.path.abspath(relative_parent)

print(parent_directory())

def str_2_int(value):
  if ',' in value:
    x = float(value.replace(',',''))
    return x
  else :
    return float(value)

salaries = pd.read_csv(parent_directory() + '/final_project/0.Ressources/05_2_salaries.csv')


c:\Users\33668\Documents\Albert school Msc1\Pandas


# Renaming Columns and Adding Total Remuneration

In [297]:


#Rename Column names because they were too long
salaries.columns = ['Time', 'Age', 'Industry', 'Job', 'Job Context', 'Salary', 'Bonus', 'Currency', 'Other Currency', 'Income Context', 'Country', 'US State', 'City', 'Experience', 'Field experience', 'Education', 'Gender', 'Race']

#Convert String to float
salaries["Salary"] = salaries["Salary"].apply(str_2_int)


#Replace NA values in Bonus by 0 and add a "Total Remuneration" Column
values = {"Bonus": 0.0}
salaries = salaries.fillna(value = values)
salaries["Total Remuneration"] = salaries["Salary"] + salaries["Bonus"]
move = salaries.pop("Total Remuneration")
salaries.insert(7,"Total Remuneration" ,move)


salaries


,Time,Age,Industry,Job,Job Context,Salary,Bonus,Total Remuneration,Currency,Other Currency,Income Context,Country,US State,City,Experience,Field experience,Education,Gender,Race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,55000.0,0.0,55000.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,54600.0,4000.0,58600.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,34000.0,0.0,34000.0,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,62000.0,3000.0,65000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,60000.0,7000.0,67000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28080,9/25/2024 19:54:18,45-54,"Accounting, Banking & Finance",Product Manager Lead,NaN,117000.0,8000.0,125000.0,USD,NaN,NaN,USA,Missouri,Remote,11 - 20 years,11 - 20 years,College degree,Woman,White
28081,9/25/2024 21:34:43,45-54,Education (Primary/Secondary),Curriculum Writer,I am a freelance contract curriculum writer fo...,70000.0,0.0,70000.0,USD,NaN,NaN,United States,South Carolina,Bennettsville,21 - 30 years,21 - 30 years,Master's degree,Woman,White
28082,9/30/2024 10:52:30,55-64,Government and Public Administration,Clerical Officer,NaN,28600.0,0.0,28600.0,EUR,NaN,NaN,Ireland,NaN,Dublin,21 - 30 years,1 year or less,"Professional degree (MD, JD, etc.)",Man,White
28083,10/4/2024 13:05:24,18-24,Computing or Tech,Software Engineering Co-Op,I was an Intern,56160.0,0.0,56160.0,USD,NaN,NaN,United States,Rhode Island,Jhonston,1 year or less,1 year or less,College degree,Man,White


# Fixing Types

In [298]:
salaries["Time"] = pd.to_datetime(salaries["Time"], format='%m/%d/%Y %H:%M:%S')

#Did that because I noticed that some values were float in Industry
salaries['Industry'] = salaries['Industry'].astype(str)

salaries.dtypes

Time                  datetime64[ns]
Age                           object
Industry                      object
Job                           object
Job Context                   object
Salary                       float64
Bonus                        float64
Total Remuneration           float64
Currency                      object
Other Currency                object
Income Context                object
Country                       object
US State                      object
City                          object
Experience                    object
Field experience              object
Education                     object
Gender                        object
Race                          object
dtype: object

# Check Missing Values

In [299]:
pair_list = salaries["Currency"].unique()

In [300]:
missing_values = salaries.isnull().sum()
missing_percentage = (missing_values / len(salaries)) * 100
missing_df = pd.DataFrame({'Missing Values': missing_values, 'Percentage (%)': missing_percentage})
missing_df[missing_df['Missing Values'] > 0].sort_values(by='Percentage (%)', ascending=False)

,Missing Values,Percentage (%)
Other Currency,27878,99.262952
Income Context,25041,89.161474
Job Context,20818,74.124978
US State,5026,17.895674
Education,222,0.790458
Race,177,0.630230
Gender,171,0.608866
City,82,0.291971
Job,1,0.003561


We can therefore see that over 99.2% of the dataset is payed in one of the currencies listed in the "Currency" column. For this reason we think it would make sense to remove the other currencies from our analysis as they would not have a significant impact on the results. We will therefore analyse salaries from the US, Great Britain, Canada, Euro Zone, Australia and New Zealand, Switzerland, South Africa, Sweden, Hong Kong and Japan. This may bias our views due to it being focused on advanced economies.   

# Get current exchange rate

In [301]:
#We are going to use Selenium here to get the exchange rate of the currencies
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ChromeService
import time

# instantiate a Chrome options object
options = webdriver.ChromeOptions()

# set the options to use Chrome in headless mode
options.add_argument("--headless=new")

# set the service to use chromedriver (for non-Mac environment especially)

service = ChromeService(executable_path='chromedriver.exe')

# initialize an instance of the chrome driver (browser) in headless mode
driver = webdriver.Chrome(service = service,
    options=options
)

# visit your target site
driver.get("https://www.oanda.com/currency-converter/live-exchange-rates/")

# extract all the product containers
rates = driver.find_elements(By.XPATH, "/html/body/div[1]/main/div/div/div[3]")

time.sleep(3)

In [302]:
def get_current_rates():
    def get_rate(rate):
        if rate == "EUR":
          return 1
        #HKD doesnt exist unfortunately and I couldn't find it so i will hard code the rate
        if rate == "HKD":
          return 0.12
        
        #Users put AUD/NZD so we chose AUD as default 
        if rate == "AUD/NZD":
          rate = "AUD"
        
        rate_pair = rate + "/EUR"
        for rate in rates:
          if rate_pair in rate.text:
            exchange_rate_path = rate.text.split(rate_pair)[1].split("\n")
            return float(exchange_rate_path[1] + exchange_rate_path[2])

    pair_list = salaries['Currency'].unique().tolist()
    exchange_rate = {}
    
    for pair in pair_list:
      if pair == "Other":
        continue
      exchange_rate[pair] = get_rate(pair)
    
    return exchange_rate


def get_total_eur(row):
    for currency_key in ['Currency', 'Other Currency']:
        currency = row[currency_key]
        if currency in rates_list:
            return row['Total Remuneration'] * rates_list[currency]
    return None  # Return None if currency is not found

rates_list = get_current_rates()


In [303]:

salaries['Total Remuneration EUR'] = salaries.apply(get_total_eur, axis=1)
salaries

,Time,Age,Industry,Job,Job Context,Salary,Bonus,Total Remuneration,Currency,Other Currency,Income Context,Country,US State,City,Experience,Field experience,Education,Gender,Race,Total Remuneration EUR
0,2021-04-27 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,55000.0,0.0,55000.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White,51304.000
1,2021-04-27 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,54600.0,4000.0,58600.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White,70613.000
2,2021-04-27 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,34000.0,0.0,34000.0,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White,31715.200
3,2021-04-27 11:02:41,25-34,Nonprofits,Program Manager,NaN,62000.0,3000.0,65000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White,60632.000
4,2021-04-27 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,60000.0,7000.0,67000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White,62497.600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28080,2024-09-25 19:54:18,45-54,"Accounting, Banking & Finance",Product Manager Lead,NaN,117000.0,8000.0,125000.0,USD,NaN,NaN,USA,Missouri,Remote,11 - 20 years,11 - 20 years,College degree,Woman,White,116600.000
28081,2024-09-25 21:34:43,45-54,Education (Primary/Secondary),Curriculum Writer,I am a freelance contract curriculum writer fo...,70000.0,0.0,70000.0,USD,NaN,NaN,United States,South Carolina,Bennettsville,21 - 30 years,21 - 30 years,Master's degree,Woman,White,65296.000
28082,2024-09-30 10:52:30,55-64,Government and Public Administration,Clerical Officer,NaN,28600.0,0.0,28600.0,EUR,NaN,NaN,Ireland,NaN,Dublin,21 - 30 years,1 year or less,"Professional degree (MD, JD, etc.)",Man,White,28600.000
28083,2024-10-04 13:05:24,18-24,Computing or Tech,Software Engineering Co-Op,I was an Intern,56160.0,0.0,56160.0,USD,NaN,NaN,United States,Rhode Island,Jhonston,1 year or less,1 year or less,College degree,Man,White,52386.048


# Check for Weird Values and Outliers

In [304]:

year = salaries['Time'].dt.year
year.unique()
# Output 2021, 2022, 2023, 2024. Nothing weird

salaries['Job Context'].unique()
# Output is okay as well

salaries['Education'].unique()
# Nothing weird here as well

previous_nb_rows = salaries.shape[0]
salaries = salaries.drop(salaries[salaries['Total Remuneration'] == 0].index)
# 18 values that wasn't important for us as there is no remuneration. We can consider those as bad data

# Dropping the NULL values in the Total Remuneration EUR column because they represent way less than 0.8% of the data. These are the values that were on "Other Currency" column and were not with the "Currency" values USD, EUR, GBP, CAD, AUD, NZD, HKD.
salaries = salaries.drop(salaries[salaries['Total Remuneration EUR'].isnull()].index)


print("We removed", previous_nb_rows - salaries.shape[0], "rows")

We removed 161 rows


In [305]:
# Function to detect outliers using IQR
def detect_outliers(dataframe, column):
    Q1 = dataframe[column].quantile(0.25)
    Q3 = dataframe[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    return dataframe[(dataframe[column] < lower_bound) | (dataframe[column] > upper_bound)]

numerical_columns = salaries.select_dtypes(exclude=['object']).columns

# Detect outliers for numerical columns
outliers_data = {}
for column in numerical_columns:
    outliers = detect_outliers(salaries, column)
    outliers_data[column] = len(outliers)

# Display the count of outliers for each numerical column
outliers_data

{'Time': 4548,
 'Salary': 1129,
 'Bonus': 3974,
 'Total Remuneration': 1561,
 'Total Remuneration EUR': 1547}

Given the nature of this dataset, the presence of outliers in price-related columns (e.g., `Salary`, `Bonus`, `Total Remuneration`,`Total Remuneration EUR`) might be due to some salaries/bonus being higher than the others. Similarly, outliers in columns like `Time` are normal because it indicates all the timestamps

For the purpose of this project, we won’t remove these outliers since they can provide insights into this variety.

# Group similar values in the same category 

In [306]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def categorize_industries(industry_list, threshold):

    categories = {}
    for industry in industry_list:
        # Find the best match for the current industry in the existing categories
        best_match = process.extractOne(industry, categories.keys(), scorer=fuzz.ratio)
        if best_match and best_match[1] >= threshold:
            # If a good match is found, assign the same category Print to check the match
            print(f"Match found: {industry} -> {best_match[0]} with score {best_match[1]}")
            categories[industry] = categories[best_match[0]]
        else:
            # Otherwise, create a new category
            categories[industry] = industry
    return categories

previous_nb = salaries['Industry'].nunique()

# Get the unique industries from the 'Industry' column
unique_industries = salaries['Industry'].dropna().unique()

# Categorize the industries
# We chose threshold 60 because the lower thresholds were giving very bad matches!
# 60 seems to be the sweet spot
industry_categories = categorize_industries(unique_industries,60)

# Map the categories back to the 'Industry' column
salaries['Industry'] = salaries['Industry'].map(industry_categories)




Match found: public library -> Public Library with score 100
Match found: University administration -> Government and Public Administration with score 62
Match found: Real Estate -> Commercial Real Estate  with score 67
Match found: Consulting -> Business or Consulting with score 62
Match found: Environmental Consulting -> Environmental regulation with score 67
Match found: Public library -> Public Library with score 100
Match found: Biotech -> Biotechnology with score 70
Match found: Public health in higher education -> Education (Higher Education) with score 67
Match found: Food manufacturing -> Engineering or Manufacturing with score 65
Match found: B2B Services  -> Business Services with score 69
Match found: Manufacturing -> Food manufacturing with score 84
Match found: Veterinary medicine -> Academic Medicine  with score 61
Match found: Health Insurance  -> Insurance with score 72
Match found: Government contractor -> Government and Public Administration with score 63
Match found

In [307]:
print(f"Unique categories in Industry column moved from {previous_nb} categories to {salaries['Industry'].nunique()} categories")


Unique categories in Industry column moved from 1210 categories to 294 categories


In [308]:
previous_nb = salaries['Country'].nunique()

# Get the unique countries from the 'Country' column
unique_countries = salaries['Country'].dropna().unique()

# Categorize the countries
# We chose threshold 80 because the lower thresholds were giving very bad matches!
# 80 seems to be the sweet spot for countries as 60 merged Myanmar and Danemark for example

# Our function will do most of the work but we will have to manually correct some countries at the end
countries = categorize_industries(unique_countries,80)

# Map the categories back to the 'Industry' column
salaries['Country'] = salaries['Country'].map(countries)



Match found: USA -> US with score 80
Match found: United Kingdom  -> United Kingdom with score 100
Match found: usa -> USA with score 100
Match found: U.S. -> US with score 80
Match found: United States  -> United States with score 100
Match found: us -> US with score 100
Match found: Usa -> USA with score 100
Match found: United states -> United States with score 100
Match found: Scotland -> Scotland  with score 100
Match found: USA  -> USA with score 100
Match found: United states  -> United States with score 100
Match found: UK  -> UK with score 100
Match found: united states -> United States with score 100
Match found: Australia -> Australia  with score 100
Match found: Uk -> UK with score 100
Match found: United States of America  -> United States of America with score 100
Match found: U.S.  -> U.S. with score 100
Match found: canada -> Canada with score 100
Match found: Canada  -> Canada with score 100
Match found: U.S> -> U.S. with score 100
Match found: US  -> US with score 100

Match found: singapore -> Singapore with score 100
Match found: usa  -> USA with score 100
Match found: Malaysia  -> Malaysia with score 100
Match found: new zealand -> New Zealand with score 100
Match found: Usat -> USA with score 86
Match found: Luxemburg -> Luxembourg with score 95
Match found: Unitef Stated -> United Stated with score 92
Match found: UA -> USA with score 80
Match found: United Stares  -> United Stares with score 100
Match found: England, United Kingdom  -> England, United Kingdom with score 100
Match found: United STates -> United States with score 100
Match found: USaa -> USA with score 86
Match found: uSA -> USA with score 100
Match found: The netherlands -> The Netherlands with score 100
Match found: america -> America with score 100
Match found: switzerland -> Switzerland with score 100
Match found: Wales (UK) -> Wales, UK with score 100
Match found: Australian  -> Australia  with score 95
Match found: uS -> US with score 100
Match found: USD -> US with score 8

In [309]:
print(f"Unique categories in Countries column moved from {previous_nb} categories to {salaries['Country'].nunique()} categories")

Unique categories in Countries column moved from 354 categories to 168 categories


In [310]:
# We chose not to put the people that work in a US company but are not in the US in the "US" category
variationsUS = [
    'United States','US','U.S.A','America','USA-- Virgin Islands','Worldwide (based in US but short term trips aroudn the world)','United States (I work from home and my clients are all over the US/Canada/PR','US of A',
    'Canada and USA','The US','USA tomorrow ',"USA, but for foreign gov't","I work for a UAE-based organization, though I am personally in the US."
]
replacementUS = 'United States of America'


variationsUK = [
    'UK',
    'Scotland ',
    'England',
    'Great Britain ',
    'Northern Ireland',
    'Britain ',
    'United Kingdom (England)',
    'Jersey, Channel islands',
    'UK for U.S. company',
    'England, United Kingdom',
    'Wales',
    'UK (England)',
    'UK, remote',
    'Wales, UK',
    'Isle of Man',
    'Northern Ireland, United Kingdom',
    'UK, but for globally fully remote company',
    'London'
]
replacementUK = 'United Kingdom'


variationsCanada = [
    'Canada, Ottawa, ontario',
    'I am located in Canada but I work for a company in the US',
    'Can'
]
replacementCanada = 'Canada'

variationsNZ = [
    'New Zealand Aotearoa',
    'Aotearoa New Zealand'
]
replacementNZ = 'New Zealand'

salaries['Country'].replace(variationsUS, replacementUS, inplace=True)
salaries['Country'].replace(variationsUK, replacementUK, inplace=True)
salaries['Country'].replace(variationsCanada, replacementCanada, inplace=True)
salaries['Country'].replace(variationsNZ, replacementNZ, inplace=True)



C:\Users\33668\AppData\Local\Temp\ipykernel_19496\3763961629.py:45: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  salaries['Country'].replace(variationsUS, replacementUS, inplace=True)


# Rearrange the order of columns

In [ ]:
salaries.pop('Total Remuneration')

column_to_move = 'Total Remuneration EUR'
target_position = 10

# Get the list of columns
columns = list(salaries.columns)
columns.remove(column_to_move)
columns.insert(target_position, column_to_move)
salaries = salaries[columns]


salaries

,Time,Age,Industry,Job,Job Context,Salary,Bonus,Currency,Other Currency,Income Context,Total Remuneration EUR,Country,US State,City,Experience,Field experience,Education,Gender,Race
0,2021-04-27 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,55000.0,0.0,USD,NaN,NaN,51304.000,United States of America,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,2021-04-27 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,54600.0,4000.0,GBP,NaN,NaN,70613.000,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,2021-04-27 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,34000.0,0.0,USD,NaN,NaN,31715.200,United States of America,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,2021-04-27 11:02:41,25-34,Nonprofits,Program Manager,NaN,62000.0,3000.0,USD,NaN,NaN,60632.000,United States of America,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,2021-04-27 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,60000.0,7000.0,USD,NaN,NaN,62497.600,United States of America,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28080,2024-09-25 19:54:18,45-54,"Accounting, Banking & Finance",Product Manager Lead,NaN,117000.0,8000.0,USD,NaN,NaN,116600.000,United States of America,Missouri,Remote,11 - 20 years,11 - 20 years,College degree,Woman,White
28081,2024-09-25 21:34:43,45-54,Education (Primary/Secondary),Curriculum Writer,I am a freelance contract curriculum writer fo...,70000.0,0.0,USD,NaN,NaN,65296.000,United States of America,South Carolina,Bennettsville,21 - 30 years,21 - 30 years,Master's degree,Woman,White
28082,2024-09-30 10:52:30,55-64,Government and Public Administration,Clerical Officer,NaN,28600.0,0.0,EUR,NaN,NaN,28600.000,Ireland,NaN,Dublin,21 - 30 years,1 year or less,"Professional degree (MD, JD, etc.)",Man,White
28083,2024-10-04 13:05:24,18-24,Computing or Tech,Software Engineering Co-Op,I was an Intern,56160.0,0.0,USD,NaN,NaN,52386.048,United States of America,Rhode Island,Jhonston,1 year or less,1 year or less,College degree,Man,White


# Questions
1. Which industry pays the most?
2. How does salary increase given years of experience?
3. How do salaries compare for the same role in different locations?
4. How much do salaries differ by gender and years of experience?
5. How do factors like race and education level correlate with salary?
6. Is there a “sweet spot” total work experience vs years in the specific field?

In [ ]:
# Define a mapping for industries into approximately 20 general categories
industry_mapping = {
    # General categories
    'Education': ['Education (Higher Education)', 'Education', 'Primary/Secondary Education', 'Research'],
    'Technology': ['Computing or Tech', 'Internet', 'Information Technology'],
    'Finance': ['Accounting, Banking & Finance', 'Investment Banking', 'Financial Services'],
    'Healthcare': ['Healthcare', 'Medical Devices', 'Pharmaceuticals', 'Biotechnology'],
    'Nonprofit': ['Nonprofits', 'Social Services', 'Charity'],
    'Government': ['Government and Public Administration', 'Military', 'Law Enforcement'],
    'Consulting': ['Consulting', 'Business Consulting', 'Management Consulting'],
    'Retail': ['Retail', 'E-commerce', 'Consumer Goods'],
    'Manufacturing': ['Manufacturing', 'Industrial Engineering', 'Machinery'],
    'Construction': ['Construction', 'Real Estate', 'Architecture'],
    'Energy': ['Energy', 'Utilities', 'Renewable Energy'],
    'Media': ['Media', 'Broadcasting', 'Publishing', 'Advertising'],
    'Transportation': ['Transportation', 'Logistics', 'Automotive'],
    'Hospitality': ['Hospitality', 'Travel', 'Tourism'],
    'Legal': ['Law', 'Legal Services', 'Compliance'],
    'Agriculture': ['Agriculture', 'Farming'],
    'Telecommunications': ['Telecommunications', 'Mobile Services'],
    'Arts & Entertainment': ['Arts', 'Entertainment', 'Film', 'Music'],
    'Food & Beverage': ['Food & Beverage', 'Wine & Spirits'],
    'Other': ['Student', 'Other', 'Self-employed', 'Social networks', 'Unknown']
}

# Function to categorize industries based on the mapping
def categorize_industry(industry):
    for category, industry_list in industry_mapping.items():
        if industry in industry_list:
            return category
    return 'Other'  # Default category if no match is found

# Apply the categorization function to create a new 'Industry Category' column
salaries['Industry'] = salaries['Industry'].apply(categorize_industry)

# Display the updated DataFrame with the new 'Industry Category' column
salaries.shape


In [ ]:
salary_average = salaries[['Industry','Total Remuneration EUR']]
list_pay_industry = salary_average.groupby("Industry").mean()
print(list_pay_industry.sort_values(by="Total Remuneration EUR", ascending= False))

In [ ]:
import seaborn
plt.figure(figsize=(12, 6))
seaborn.boxplot(x=salaries['Experience'], y=salaries['Total Remuneration EUR'], palette='Blues')